#### All information is scraped using the website camelcamelcamel.com and amazon.com

In [1]:
from urllib.request import urlopen    #Helps open up URLs
import re                             #Allows for the use of regular expressions

In [2]:
import tweepy #module for interacting with the Twitter API

CONSUMER_KEY = ''#keep the quotes, replace this with your consumer key
CONSUMER_SECRET = ''#keep the quotes, replace this with your consumer secret key
ACCESS_TOKEN = ''#keep the quotes, replace this with your access token
ACCESS_SECRET = ''#keep the quotes, replace this with your access token secret
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth) #this creates the API object

In [11]:
# this module allows us to open a file from a URL:
import urllib.request 

# this module allows us to parse an XML file, similar to Beautiful Soup
# read more here: https://docs.python.org/3.0/library/xml.dom.minidom.html
from xml.dom import minidom 

# open the URL:
from urllib.request import Request, urlopen
url="https://camelcamelcamel.com/top_drops/feed?bn=electronics&t=absolute&i=7&s=absolute&d=0.01&"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

web_byte = urlopen(req).read()

open_url = web_byte.decode('utf-8')

# parse the XML document:
xmldoc = minidom.parseString(open_url)

# print the page source:
print(xmldoc.toprettyxml())

<?xml version="1.0" ?>
<rss version="2.0">
	
  
	<channel>
		
    
		<title>Top Amazon Price Drops</title>
		
    
		<link>https://camelcamelcamel.com/top_drops?bn=electronics&amp;d=0.01&amp;i=7&amp;s=absolute&amp;t=absolute&amp;</link>
		
    
		<description>Amazon top price drops list by camelcamelcamel.com</description>
		
    
		<language>en-us</language>
		
    
		<pubDate>Sat, 06 Jun 2020 21:00:34 PDT</pubDate>
		
    
		<lastBuildDate>Sat, 06 Jun 2020 21:00:34 PDT</lastBuildDate>
		
    
		<generator>camelcamelcamel.com by Cosmic Shovel</generator>
		
    
		<managingEditor>dan@shup.com (Daniel Green)</managingEditor>
		
    
		<webMaster>dan@shup.com (Daniel Green)</webMaster>
		
    
		<item>
			
      
			<title>Roku Express HD Streaming Media Player 2019 (Renewed) - down 95.30% ($830.14) to $40.96 from $871.10</title>
			
      
			<link>https://camelcamelcamel.com/product/B07ZQH6YGY</link>
			
      
			<description>&lt;p&gt;Useful URLs: &lt;a href=&quot;https://camelcamelc

In [12]:
def getBS(linkReq):
    req = requests.get(linkReq) #Requests the link from the getLink() function
    reqText = req.text
    webSoup = BeautifulSoup(reqText, 'html.parser')
    return webSoup

In [13]:
saleArray = []
x = 1

def getSale():
    global x
    element = xmldoc.getElementsByTagName('title')[x]
    sale = element.firstChild.toxml()
    
    if sale in saleArray: #Checks if the sale has been tweeted out recently. If it has, go to the next sale.
        x = x+1
        element = xmldoc.getElementsByTagName('title')[x]
        sale = element.firstChild.toxml()
        saleArray.append(sale)
    else: #Resets it back to the first sale
        x = 1
        saleArray.append(sale)
    global saleText
    sale1 = re.sub('(.*?) - ', ' ', sale) #Retrieves the string with the description
    saleText = sale[:30] + sale1
    return saleText

In [14]:
def getLink():
    global x
    description = xmldoc.getElementsByTagName('description')[x] #Retrieves location of the link
    descriptionText = description.firstChild.toxml()
    global linkText
    #Extracts just the link
    linkText = re.search('&lt;p&gt;Useful URLs: &lt;a href=&quot;(.*)&quot;&gt;Amazon.com&lt;/a&gt;,', descriptionText)

In [15]:
def getPic():
    try:
        imgText = getBS(linkText.group(1)).find_all(id="landingImage") #Finds the main image of the item
        imgLink = re.search('{"(.*)":', str(imgText)) #Extracts link to image
        itmImage = urllib.request.urlretrieve(imgLink.group(1), "item_image.jpg") #Saves the image
        return itmImage[0] #Returns the name of the saved image
    except: #If the first image is not available, then the process is repeated with another image
        imgText = getBS(linkText.group(1)).find_all(id="comparison_image") #If the main image can't be found, finds another one
        imgLink = re.search('src="(.*)">', str(imgText))
        itmImage = urllib.request.urlretrieve(imgLink.group(1), "item_image.jpg")
        return itmImage[0]

In [16]:
def tweetSale():
    getSale() #Finds the sale
    getLink() #Finds the link
    saleDesc = saleText, linkText.group(1) #Creates the text of the tweet
    
    #Tweets the description with the picture. In the case of an error, tweet just the description.
    try: 
        api.update_with_media(getPic(), saleDesc)
    except: 
        api.update_status(saleDesc)

In [ ]:
import time

#Tweets sale once a day
while True:
    tweetSale()
    
    time.sleep(86400)

In [ ]:
def replySale():
    pgNum = 1
    itemLink = None
    for i in range(0,25): #Cycles through at most 25 of the pages on camelcamelcamel.com
        if itemLink is None:
            try:
                #Requests the page and converts it into BeautifulSoup format
                saleList = "https://camelcamelcamel.com/top_drops?bn=electronics&t=absolute&i=7&s=absolute&d=0.01&p="

                searchText = getBS(saleList + str(pgNum)).find_all("h3") #Finds the list sales for each page
                itemRequest = re.findall('<h3>(.*)</a>', str(searchText)) #Extracts the description for all the items

                x = 0 #Starts at the first item
                if message_text.lower() in itemRequest[x].lower(): #Checks if the message is in the current item
                    itemLink = re.search('href="(.*)" onclick', itemRequest[x].lower()) #Finds a link to a more detailed page
                else: #If the message is not in the item go to the next item
                    while message_text.lower() not in itemRequest[x].lower():
                        x = x + 1
                        if message_text.lower() in itemRequest[x].lower():
                            itemLink = re.search('href="(.*)" onclick', itemRequest[x].lower())
            except:
                pgNum = pgNum + 1
                pass
    if itemLink is None:
        api.update_status("Sorry, I couldn't find anything @" + name)
    else:
        #Finds the link to the Amazon page
        amzReply = getBS(itemLink.group(1)).find(title="View the product page at Amazon")
        amzReplyLink = re.findall('<a href="(.*)" onclick=', str(amzReply))
        
        #Retrieves the title for the item
        replyTitle = getBS(itemLink.group(1)).find(property="og:title")
        amzReplyTitle = re.findall('meta content="(.*)" property=', str(replyTitle))
        shortenedTitle = amzReplyTitle[0]
        
        #Retrieves the image from the Amazon page
        imgReplyText = getBS(amzReplyLink[0]).find_all(id="landingImage")
        imgReplyLink = re.findall('{"(.*)":', str(imgReplyText))
        urllib.request.urlretrieve(imgReplyLink[0], "reply_image.jpg")
        
        replyDescription = shortenedTitle[:40] + " " + amzReplyLink[0] + " @" + name #Creates the description to be tweeted
        api.update_with_media("reply_image.jpg", replyDescription)

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self,status):     # upon detecting a new tweet (status)
        global message_text
        global name
        name = status.user.screen_name # get the username of the account sending the tweet
        id_str = status.id_str # get the unique ID of tweet
        text = status.text # get the text of the tweet
        message_text = text.split(" ",1)[1] # get the main body of the tweet following the username, @YourHandleHere
        message_text.lower()
        replySale()
        
    def on_error(self,status_code): # if Twitter tells us we are tweeting too fast 
        if status_code == 420:      # this will result in us getting a status code of 420
            return False            # if so, end the session

In [ ]:
myStreamListener = MyStreamListener() # create an instance of the MyStreamListener class with the name myStreamListener
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener) #pass your API key and stream to tweepy

myStream.filter(track=['@TechTricksCS'])

In [ ]:
myStream.disconnect()